In [ ]:
from flask import Flask, render_template_string, request
from flask_ngrok import run_with_ngrok
import requests
import pandas as pd
import numpy as np
df = pd.read_csv('client_portfolio_data.csv')
import json
json_list_str = df.tail(1).to_json(orient='records')
single_json_obj = json.loads(json_list_str)[0]
client_details = json.dumps(single_json_obj)
app = Flask(__name__)

# Replace with your actual API Gateway endpoint
API_GATEWAY_URL = f"https://im9cd3y00h.execute-api.us-east-1.amazonaws.com/generatePersonalisedInsights?client_details={client_details}"

# UI route
@app.route('/', methods=['GET', 'POST'])
def index():
    insights = None
    error = None
    
    if request.method == 'POST':
        client_name = request.form.get('client_name')

        try:
            response = requests.post(API_GATEWAY_URL, json={"name": client_name})
            response.raise_for_status()
            insights = response.json()
        except Exception as e:
            error = str(e)

    html = """
    <!DOCTYPE html>
    <html>
    <head>
        <title>Personalized Insights Viewer</title>
    </head>
    <body>
        <h2>Fetch Personalized Insights</h2>
        <form method="POST">
            <label for="client_name">Client Name:</label>
            <input type="text" id="client_name" name="client_name" required>
            <button type="submit">Get Insights</button>
        </form>

        {% if insights %}
            <h3>Insights for {{ insights.name }}</h3>
            <pre>{{ insights | tojson(indent=2) }}</pre>
        {% elif error %}
            <p style="color:red;">Error: {{ error }}</p>
        {% endif %}
    </body>
    </html>
    """
    return render_template_string(html, insights=insights, error=error)

app.run()


/home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/pandas/core/computation/expressions.py:21: UserWarning: Pandas requires version '2.8.4' or newer of 'numexpr' (version '2.7.3' currently installed).
  from pandas.core.computation.check import NUMEXPR_INSTALLED


 * Serving Flask app '__main__'
 * Debug mode: off


 * Running on http://127.0.0.1:5000
Press CTRL+C to quit
